In [5]:
import numpy as np
from scipy.stats import norm

In [8]:
def IG_generator(delta, gamma, M, N):
  """
  Generate MxN samples from the Inverse Gaussian (IG) distribution IG(delta,sqrt(alpha**2 - beta**2)).

  Parameters:
  - delta: parameter of the IG distribution
  - gamma: parameter of the IG distribution
  - M: number of rows (samples)
  - N: number of columns (samples)

  Returns:
  - IG: MxN array of samples from the IG distribution
  """
  # Generate chi-squared samples with degree of freedom = 1.
  V = np.random.chisquare(1, (M, N))

  # Intermediate computations for generating IG samples
  x1 = (delta / gamma) * np.ones((M, N)) + (1 / (2 * gamma ** 2)) * (V + np.sqrt(4 * gamma * delta * V + V ** 2))
  x2 = (delta / gamma) * np.ones((M, N)) + (1 / (2 * gamma ** 2)) * (V - np.sqrt(4 * gamma * delta * V + V ** 2))

  # Generate uniform samples
  Y = np.random.uniform(0, 1, (M, N))

  # Compute probabilities
  p1 = (delta * np.ones((M, N))) / (delta * np.ones((M, N)) + gamma * x1)
  p2 = np.ones((M, N)) - p1

  # Generate binary indicator matrix
  C = (Y < p1)

  # Generate IG samples
  IG = C * x1 + (np.ones((M, N)) - C) * x2
  return IG

def MC_call_on_min_NIG_pricer(S0, K, r, T, alpha, beta, delta, DELTA, B, N, M, alpha_conf, seed):
  """
  Compute the price of a call on min option under normal inverse Gaussian (NIG) for a system of d stocks.

  Parameters:
  - S0: vector of initial stock prices
  - K: strike price
  - r: risk-free interest rate
  - T: maturity time
  - alpha: parameter of the NIG distribution
  - beta: parameter of the NIG distribution
  - delta: parameter of the NIG distribution
  - DELTA: covariance matrix
  - B: number of batches
  - N: number of time steps of forward Euler discretization
  - M: number of MC sample paths
  - alpha_conf: confidence level
  - seed: random seed

  Returns:
  - price_estimate: estimated price of the option using Monte Carlo
  - MC_stat_error: statistical error of the Monte Carlo estimation
  """
  np.random.seed(seed)
  dimension = len(S0)  # number of stocks
  price_estimates_per_batch = np.zeros(B)  # MC price estimates per batch
  price_stds_per_batch = np.zeros(B)  # MC standard deviation per batch
  N = 1  # Exact simulation scheme
  dt = T / N  # size of time sub-interval between two consecutive increments
  L = np.linalg.cholesky(DELTA)
  gamma = np.sqrt(alpha ** 2 - np.dot(beta, np.dot(DELTA, beta)))  # constant used in intermediate computations to ease the notation
  mu = - delta * (np.sqrt(alpha ** 2 - beta ** 2) - np.sqrt(alpha ** 2 - (beta + 1) ** 2))  # martingale correction term for d stocks.

  for b in range(B):
    X = np.zeros((M, N + 1, dimension))  # Contains all sample paths of each time step for each stock: M_paths x N_steps x d
    Z = np.random.multivariate_normal(mean=np.zeros(dimension), cov=np.identity(dimension), size=(M, N))  # samples from independent standard normal distribution
    IG = IG_generator(delta, gamma, M, N)  # generation of the common inverse Gaussian clock for all stocks at each sample path and time step.

    for n in range(N):
      for m in range(M):
        X[m, n + 1, :] = X[m, n, :] + (r + mu) * dt + IG[m][n] * DELTA @ beta + np.sqrt(IG[m][n]) * L @ Z[m][n]  # Increment of the NIG process

    XT = X[:, -1, :]  # asset log-price at the final time for each sample path
    ST = S0 * np.exp(XT)  # Stock prices at maturity
    # Compute payoffs for each path
    payoff_evals = np.exp(-r * T) * np.maximum(np.min(ST, axis=1) - K, 0)
    price_estimates_per_batch[b] = np.mean(payoff_evals)  # MC price estimate per batch
    price_stds_per_batch[b] = np.std(payoff_evals)  # MC standard deviation per batch

  price_estimate = np.mean(price_estimates_per_batch)  # Final MC estimate
  C_alpha = norm.ppf(1 - alpha_conf / 2)
  MC_stat_error = C_alpha * np.mean(price_stds_per_batch) / np.sqrt(M * B)  # Final MC statistical error
  return price_estimate, MC_stat_error

def MC_basket_put_NIG_pricer(S0, K, r, T, alpha, beta, delta, DELTA, B, N, M, alpha_conf, seed):
  """
  Compute the price of a basket put option under normal inverse Gaussian (NIG) for a system of d stocks.

  Parameters:
  - S0: vector of initial stock prices
  - K: strike price
  - r: risk-free interest rate
  - T: maturity time
  - alpha: parameter of the NIG distribution
  - beta: parameter of the NIG distribution
  - delta: parameter of the NIG distribution
  - DELTA: covariance matrix
  - B: number of batches
  - N: number of time steps of forward Euler discretization
  - M: number of MC sample paths
  - alpha_conf: confidence level
  - seed: random seed

  Returns:
  - price_estimate: estimated price of the option using Monte Carlo
  - MC_stat_error: statistical error of the Monte Carlo estimation
  """
  np.random.seed(seed)
  dimension = len(S0)  # number of stocks
  price_estimates_per_batch = np.zeros(B)  # MC price estimates per batch
  price_stds_per_batch = np.zeros(B)  # MC standard deviation per batch
  N = 1  # Exact simulation scheme
  dt = T / N  # size of time sub-interval between two consecutive increments
  L = np.linalg.cholesky(DELTA)
  gamma = np.sqrt(alpha ** 2 - np.dot(beta, np.dot(DELTA, beta)))  # constant used in intermediate computations to ease the notation
  mu = - delta * (np.sqrt(alpha ** 2 - beta ** 2) - np.sqrt(alpha ** 2 - (beta + 1) ** 2))  # martingale correction term for d stocks.

  for b in range(B):
    X = np.zeros((M, N + 1, dimension))  # Contains all sample paths of each time step for each stock: M_paths x N_steps x d
    Z = np.random.multivariate_normal(mean=np.zeros(dimension), cov=np.identity(dimension), size=(M, N))  # samples from independent standard normal distribution
    IG = IG_generator(delta, gamma, M, N)  # generation of the common inverse Gaussian clock for all stocks at each sample path and time step.

    for n in range(N):
      for m in range(M):
        X[m, n + 1, :] = X[m, n, :] + (r + mu) * dt + IG[m][n] * DELTA @ beta + np.sqrt(IG[m][n]) * L @ Z[m][n]  # Increment of the NIG process

    XT = X[:, -1, :]  # asset log-price at the final time for each sample path
    ST = S0 * np.exp(XT)  # Stock prices at maturity
    # Compute payoffs for each path
    payoff_evals = np.exp(-r * T) * np.maximum(K - np.mean(ST, axis=1), 0)
    price_estimates_per_batch[b] = np.mean(payoff_evals)  # MC price estimate per batch
    price_stds_per_batch[b] = np.std(payoff_evals)  # MC standard deviation per batch

  price_estimate = np.mean(price_estimates_per_batch)  # Final MC estimate
  C_alpha = norm.ppf(1 - alpha_conf / 2)
  MC_stat_error = C_alpha * np.mean(price_stds_per_batch) / np.sqrt(M * B)  # Final MC statistical error
  return price_estimate, MC_stat_error

# Pricing call on min options

In [4]:
############### Model and payoff parameters ###############
#Payoff Parameters
dimension = 4
S0 = 100 * np.ones(dimension)
K = 100
r = 0
T = 1
#NIG Model Parameters
alpha = 30
beta = -3 * np.ones(dimension)
delta = 0.5
DELTA = np.identity(dimension)
############### MC parameters ###############
B = 1 # number of batches
N = 1 # exact simulation scheme
M = 10**5 # number of MC sample paths
alpha_conf = 0.05 # confidence level for MC statistical error estimation
seed = 100 # random seed for reproducibility of results.
MC_price_estimate, MC_stat_error = MC_call_on_min_NIG_pricer(S0, K, r, T, alpha, beta, delta, DELTA, B, N, M, alpha_conf, seed)
print("MC price estimate =", round(MC_price_estimate, 5),"\nMC relative statistical error = ", round(MC_stat_error / MC_price_estimate,4) )

MC price estimate = 0.17764 
MC relative statistical error =  0.0353


# Pricing basket put options

In [10]:
############### Model and payoff parameters ###############
#Payoff Parameters
dimension = 4
S0 = 100 * np.ones(dimension)
K = 100
r = 0
T = 1
#NIG Model Parameters
alpha = 30
beta = -3 * np.ones(dimension)
delta = 0.5
DELTA = np.identity(dimension)
############### MC parameters ###############
B = 1 # number of batches
N = 1 # exact simulation scheme
M = 10**5 # number of MC sample paths
alpha_conf = 0.05 # confidence level for MC statistical error estimation
seed = 100 # random seed for reproducibility of results.
MC_price_estimate, MC_stat_error = MC_basket_put_NIG_pricer(S0, K, r, T, alpha, beta, delta, DELTA, B, N, M, alpha_conf, seed)
print("MC price estimate =", round(MC_price_estimate, 5),"\nMC relative statistical error = ", round(MC_stat_error / MC_price_estimate,4) )

MC price estimate = 2.65608 
MC relative statistical error =  0.009
